# Microtask2
>Like Microtask 1, but now using [Pandas](http://pandas.pydata.org/)
>>Microtask 1: Produce a notebook showing (and producing) a list with the activity per quarter: number of new committers, submitters of issues, and submitters of pull/merge requests, number of items (commits, issues, pull/merge requests), number of repositories with new items (all of this per quarter) as a table and as a CSV file. Use plain Python3 (eg, no Pandas) for this.

## Quarters
The quarters are defined using the following dictionary where the **key corresponds to the month** and **value corresponds to the quarter**  :
```
quarter={1:'Q1', 2:'Q1', 3:'Q1', 4:'Q2', 5:'Q2', 6:'Q2', 7:'Q3', 8:'Q3', 9:'Q3', 10:'Q4', 11:'Q4', 12:'Q4'}
```

In [1]:
import json
from datetime import datetime 
import pandas as pd
import csv
import requests
from tabulate import tabulate

## Start and end year for the analysis
The startyear and endyear for a repository are obtained from the Github API using the [requests].(http://www.python-requests.org/en/latest/)
module
- startyear : year of repository creation.
- endyear : when the repository was last updated.

The steps are :
1. Call the API
1. Assuming the API returns a JSON, parse the JSON object into a Python dict using json.loads function
1. Loop through the dict to extract information.

`if(Response.ok)`: will help help you determine if your API call is successful (Response code - 200)

`Response.raise_for_status()` will help you fetch the http code that is returned from the API.

In [2]:
url = 'https://api.github.com/repos/tensorflow/datasets' # url of repo to be analysed
info = requests.get(url)
if(info.ok):
    startyear = datetime.strptime(json.loads(info.text)['created_at'], "%Y-%m-%dT%H:%M:%SZ").year
    endyear = datetime.strptime(json.loads(info.text)['updated_at'], "%Y-%m-%dT%H:%M:%SZ").year
else : 
    info.raise_for_status()

### Create a dictionary mapping months to quarters, and a list indexes which is as shown below

In [3]:
indexes = []
for year in range(startyear, endyear+1):
    for quarter in ['Q1', 'Q2', 'Q3', 'Q4']:
        indexes.append(str(year) + '_' + quarter)
        
# dictionary mapping month to quarter
quarter = {1:'Q1', 2:'Q1', 3:'Q1', 4:'Q2', 5:'Q2', 6:'Q2', 7:'Q3', 8:'Q3', 9:'Q3', 10:'Q4', 11:'Q4', 12:'Q4'}

## Functions for isssues, prs and commits

#### Adding functionality to code
The three functions below for the three categories can be merged into a single function - having category and date fields as paramters, and thus the code can be made more functional, but for clarity I have described them seperately here.
### 1) Creating a dataframe issue_df for issues
issue_df will have
- index (or quarter),
- #issues(number of issues opened in that quarter),
- users(a list of issue contributors in that quarter),
- #new_users(number of new contributors) in that quarter
as its columns. <br>

The function issue_:
```def issues_(issue_data):```
populates the issue_df dataframe as shown, **except** the **'#new_users'** column.

In [4]:
#creating the issue_df dataframe
issue_df = pd.DataFrame(columns = ['index', '#issues', 'users', '#new_users'])
issue_df['index'] = indexes
issue_df['users'] = [[] for _ in range(len(indexes))]
issue_df['#new_users'] = 0
issue_df['#issues'] = 0

# function to populate the issue_df dataframe (leaving the #new_users column for now)
def issues_(issue_data):
    created = datetime.strptime(issue_data['created_at'], "%Y-%m-%dT%H:%M:%SZ")
    index = str(created.year) + "_" + quarter[created.month]
    issue_df.loc[issue_df['index']==index, 'users'].apply(lambda x: x.append(issue_data['user_data']['login']))
    issue_df.loc[issue_df['index']==index, '#issues'] = issue_df.loc[issue_df['index']==index, '#issues'].apply(lambda x : x+1)

### 2) Creating the dataframes pr_df in the same way
and similarly the function ```def pr_(pr_data):```

In [5]:
#creating the pr_df dataframe
pr_df = pd.DataFrame(columns = ['index', '#prs', 'users', '#new_users'])
pr_df['index'] = indexes
pr_df['users'] = [[] for _ in range(len(indexes))]
pr_df['#new_users'] = 0
pr_df['#prs'] = 0

# function to populate the pr_df dataframe (leaving the #new_users column for now)
def pr_(pr_data):
    created = datetime.strptime(pr_data['created_at'], "%Y-%m-%dT%H:%M:%SZ")
    index = str(created.year) + "_" + quarter[created.month]
    pr_df.loc[pr_df['index']==index, 'users'].apply(lambda x: x.append(pr_data['user_data']['login']))
    pr_df.loc[pr_df['index']==index, '#prs'] = pr_df.loc[pr_df['index']==index, '#prs'].apply(lambda x : x+1)

### 3) Creating the dataframes commit_df in the same way
and similarly the function ```def commits_(cdata):```

In [6]:
commit_df = pd.DataFrame(columns = ['index', '#commits', 'commiters', '#new_commiters'])
commit_df['index'] = indexes
commit_df['commiters'] = [list() for _ in range(len(indexes))]
commit_df['#new_commiters'] = 0
commit_df['#commits'] = 0

# function to populate the commit_df dataframe (leaving the #new_commiters column for now)
def commits_(cdata):
    commit_date = datetime.strptime(cdata['CommitDate'],"%a %b %d %H:%M:%S %Y %z")
    index = str(commit_date.year) + "_" + quarter[commit_date.month]
    commit_df.loc[commit_df['index']==index, 'commiters'].apply(lambda x : x.append(cdata['Author']))
    commit_df.loc[commit_df['index']==index, '#commits'] = commit_df.loc[commit_df['index']==index, '#commits'].apply(
                                                                                         lambda x :  x+1)

### Reading the json file and calling the functions for corresponding item category to add it to its respective dataframe

In [7]:
with open("./tf_analysis.json") as file:
    for line in file:
        line = json.loads(line)
        if(line['category']=='commit'):
            commits_(line['data'])
        elif(line['category']=='pull_request'):
            pr_(line['data'])
        else:
            issues_(line['data'])

### Filling the #new_users column in all the three dataframes

For the initial row (initial quarter), #new_users = length of the set of users for that quarter.

In [8]:
commit_df.loc[0, "#new_commiters"] = len(set(commit_df.loc[0,"commiters"]))
pr_df.loc[0, "#new_users"] = len(set(pr_df.loc[0,"users"]))
issue_df.loc[0, "#new_users"] = len(set(issue_df.loc[0,"users"]))

For each subsequent row (subsequent quarter):<br>
- #new_users = length of (set(current_users) - set(contributors_uptil))
- contributors_uptil set would be updated by adding the current_users to it if not present

In [9]:
commiters_uptil = set(commit_df.loc[0,'commiters'])
for i in range(1, len(indexes)):
    current_commiters = set(commit_df.loc[i,'commiters'])
    commit_df.loc[i,'#new_commiters'] = len(current_commiters.difference(commiters_uptil))
    commiters_uptil.update(current_commiters)
    
issue_contrib_uptil = set(issue_df.loc[0,'users'])
for i in range(1, len(indexes)):
    current_contributors = set(issue_df.loc[i,'users'])
    issue_df.loc[i,'#new_users'] = len(current_contributors.difference(issue_contrib_uptil))
    issue_contrib_uptil.update(current_contributors)
    
pr_users_uptil = set(pr_df.loc[0,'users'])
for i in range(1, len(indexes)):
    current_users = set(pr_df.loc[i,'users'])
    pr_df.loc[i,'#new_users'] = len(current_users.difference(pr_users_uptil))
    pr_users_uptil.update(current_users)

### Printing the commit_df

In [10]:
commit_df.head(8)

,index,#commits,commiters,#new_commiters
0,2018_Q1,0,[],0
1,2018_Q2,0,[],0
2,2018_Q3,10,"[Ryan Sepassi <rsepassi@google.com>, Ryan Sepa...",3
3,2018_Q4,272,"[Ryan Sepassi <rsepassi@google.com>, Ryan Sepa...",9
4,2019_Q1,414,"[Marcin Michalski <michalski@google.com>, Marc...",30
5,2019_Q2,0,[],0
6,2019_Q3,0,[],0
7,2019_Q4,0,[],0


### Creating a final df by merging all three commit_df, issue_df, pr_df on the column index using [pandas.DataFrame.merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

In [11]:
issue_df.drop(['users'], axis = 1, inplace = True)
pr_df.drop(['users'], axis = 1, inplace = True)
commit_df.drop(['commiters'], axis = 1, inplace = True)
pr_df.rename({'#new_users':"#NewPRSubmitters"}, axis = 1, inplace = True)
issue_df.rename({'#new_users':"#NewIssueSubmitters"}, axis = 1, inplace = True)
commit_df.rename({'#new_commiters': '#NewCommiters'}, axis = 1, inplace = True)


df = issue_df.merge(pr_df, on = "index", how = "inner")
df = df.merge(commit_df, on = "index", how = "inner")
df

,index,#issues,#NewIssueSubmitters,#prs,#NewPRSubmitters,#commits,#NewCommiters
0,2018_Q1,0,0,0,0,0,0
1,2018_Q2,0,0,0,0,0,0
2,2018_Q3,4,3,1,1,10,3
3,2018_Q4,7,6,1,1,272,9
4,2019_Q1,312,71,175,34,414,30
5,2019_Q2,0,0,0,0,0,0
6,2019_Q3,0,0,0,0,0,0
7,2019_Q4,0,0,0,0,0,0


### Writing datframe to csv, and printing it a table
We can use the [tabulate](https://pypi.org/project/tabulate/) module.
The dataframe can also be printed using df.valus

In [12]:
df.to_csv("./tfdata_pandas.csv", index=False)
print(tabulate(df, headers='keys', tablefmt='psql'))

+----+---------+-----------+-----------------------+--------+--------------------+------------+-----------------+
|    | index   |   #issues |   #NewIssueSubmitters |   #prs |   #NewPRSubmitters |   #commits |   #NewCommiters |
|----+---------+-----------+-----------------------+--------+--------------------+------------+-----------------|
|  0 | 2018_Q1 |         0 |                     0 |      0 |                  0 |          0 |               0 |
|  1 | 2018_Q2 |         0 |                     0 |      0 |                  0 |          0 |               0 |
|  2 | 2018_Q3 |         4 |                     3 |      1 |                  1 |         10 |               3 |
|  3 | 2018_Q4 |         7 |                     6 |      1 |                  1 |        272 |               9 |
|  4 | 2019_Q1 |       312 |                    71 |    175 |                 34 |        414 |              30 |
|  5 | 2019_Q2 |         0 |                     0 |      0 |                  0 |      